In [1]:
import json
import requests
import os
from dotenv import load_dotenv
import math
import pandas as pd
from pandas import json_normalize 


load_dotenv(verbose=True)

True

In [16]:
api_key = os.getenv("CRUNCHBASE_API_KEY") # extract API key 
base_url = "https://api.crunchbase.com/api/"
params = {"user_key" : api_key}

Organization download

In [17]:
query_swiss = {
    "field_ids": [
        "name",
        #"created_at",
        "short_description",
        "location_identifiers",
        #"funding_stage",
        #"funding_total",
        "categories",
        "category_groups",
        "facet_ids",
        "twitter",
        "rank_org"
        
    ],
    "limit": 1000,
    "query": [
        {
          "type": "predicate",
          "field_id": "location_identifiers",
          "operator_id": "includes",
          "values": ["Switzerland"]
        },
    ]
}

r = requests.post(f"https://api.crunchbase.com/api/v4/searches/organizations", json=query_swiss, params=params)

In [18]:
query = {
    "field_ids": [
        "name",
        "short_description",
        "location_identifiers",
        "categories",
        "category_groups",
        "facet_ids",
        "twitter",
        "rank_org"
        
    ],
    "limit": 1000,
}

r = requests.post(f"https://api.crunchbase.com/api/v4/searches/organizations", json=query, params=params)

In [19]:
json_data = json.loads(r.text)

In [20]:
df = pd.json_normalize(json_data['entities'])
#df.head()

## Data Cleaning

In [21]:
# remove useless columns
df = df.drop(["uuid", "properties.identifier.image_id", "properties.identifier.uuid", "properties.identifier.entity_def_id"], axis=1) 

# rename columns
df = df.rename(columns = {'properties.name': 'name', 'properties.category_groups':'category_groups'})

In [22]:
if (df["name"] == df["properties.identifier.value"]).all() == True: # properties.identifier.value does not bring new info
    
    df = df.drop(["properties.identifier.value"], axis=1)

In [23]:
df.sort_values("properties.rank_org").head(10)

,name,properties.facet_ids,properties.identifier.permalink,category_groups,properties.short_description,properties.rank_org,properties.categories,properties.location_identifiers,properties.twitter.value
0,SoftBank,"[investor, company, rank]",softbank,"[{'entity_def_id': 'category_group', 'permalin...","SoftBank provides fixed-line, mobile telephony...",1,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'tokyo-tokyo', 'uuid': '127ba62...",https://twitter.com/softbank_group
1,Crowdcube,"[investor, company, rank]",crowdcube,"[{'entity_def_id': 'category_group', 'permalin...",Crowdcube is an equity crowdfunding platform f...,2,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'exeter-devon', 'uuid': 'c5f8ba...",https://twitter.com/Crowdcube
2,Compass,"[company, rank]",compassinc,"[{'entity_def_id': 'category_group', 'permalin...",Compass is a real estate technology company th...,3,"[{'entity_def_id': 'category', 'permalink': 'p...","[{'permalink': 'new-york-new-york', 'uuid': 'd...",https://twitter.com/Compass
3,Klarna,"[investor, company, rank]",klarna,"[{'entity_def_id': 'category_group', 'permalin...",Klarna is an eCommerce payment solutions platf...,4,"[{'entity_def_id': 'category', 'permalink': 'e...","[{'permalink': 'stockholm-stockholms-lan', 'uu...",http://twitter.com/klarna
4,Zoom,"[investor, company, rank]",zoom-video-communications,"[{'entity_def_id': 'category_group', 'permalin...",Zoom is a software company that offers a commu...,5,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'san-jose-california', 'uuid': ...",http://twitter.com/zoom_us
5,BetterUp,"[company, rank]",betterup,"[{'entity_def_id': 'category_group', 'permalin...",BetterUp is a mobile-based leadership developm...,6,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'san-francisco-california', 'uu...",https://www.twitter.com/betterup
6,Citigroup,"[investor, company, rank]",citigroup,"[{'entity_def_id': 'category_group', 'permalin...",Citigroup is a diversified financial services ...,7,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'new-york-new-york', 'uuid': 'd...",https://twitter.com/citi
7,Ant Group,"[investor, company, rank]",ant-group,"[{'entity_def_id': 'category_group', 'permalin...",Ant Group strives to enable all consumers and ...,8,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'hangzhou-zhejiang', 'uuid': '4...",https://twitter.com/antgroup
8,Intel,"[investor, company, rank]",intel,"[{'entity_def_id': 'category_group', 'permalin...","Intel designs, manufactures, and sells integra...",9,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'santa-clara-california', 'uuid...",http://twitter.com/intel
9,UiPath,"[company, rank]",uipath,"[{'entity_def_id': 'category_group', 'permalin...",UiPath is a software company developing roboti...,10,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'new-york-new-york', 'uuid': 'd...",http://www.twitter.com/uipath


In [24]:
df[["name", "properties.short_description"]].head(10)

,name,properties.short_description
0,SoftBank,"SoftBank provides fixed-line, mobile telephony..."
1,Crowdcube,Crowdcube is an equity crowdfunding platform f...
2,Compass,Compass is a real estate technology company th...
3,Klarna,Klarna is an eCommerce payment solutions platf...
4,Zoom,Zoom is a software company that offers a commu...
5,BetterUp,BetterUp is a mobile-based leadership developm...
6,Citigroup,Citigroup is a diversified financial services ...
7,Ant Group,Ant Group strives to enable all consumers and ...
8,Intel,"Intel designs, manufactures, and sells integra..."
9,UiPath,UiPath is a software company developing roboti...


In [25]:
category_groups = df["category_groups"].values
i = 0

for line in category_groups:
    
    if issubclass(type(line), float) == False:
        df["category_groups"].values[i] = [ x.get('value') for x in line]
    
    i = i+1

### Graph

In [74]:
df_graph = df[['name', 'category_groups']]
df_graph.head()

,name,category_groups
0,SoftBank,"[Financial Services, Hardware, Internet Servic..."
1,Crowdcube,"[Financial Services, Lending and Investments, ..."
2,Compass,[Real Estate]
3,Klarna,"[Commerce and Shopping, Financial Services, Pa..."
4,Zoom,"[Hardware, Information Technology, Internet Se..."


In [85]:
df_columns = list(df_graph.columns)
df_columns.remove('name')
df_columns = df_columns[0]

df_columns

'category_groups'

In [86]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite

B = nx.Graph()

In [89]:
for index, value in df_graph.iterrows():
    
    name = value['name']
    other = value[df_columns]
    
    if issubclass(type(line), float) == False:
        
    
    B.add_node(name, bipartite=0)
    
    for x in other:
        B.add_node(x, bipartite=1)
        B.add_edge(name, x)


TypeError: 'float' object is not iterable

In [91]:
other

nan